In [1]:
from stata_preprocessing import *
from llm_simulation import *
from load_environmental_distributions import load_environmental_distribution
from visualise_responses import *
from fine_tuning_prompts import *
import os
import random

In [2]:
age_groups = ["18-34", "35-54", "55 or older"]
base_json_filepath = "C:\\Users\\haoch\\Documents\\COMP0190\\Data\\COMP0191-MSc-Project-Code\\Synthetic-Responses-JSON\\Hypothesis-2"

In [3]:
stata_demo_question_filepath = "C:\\Users\\haoch\\Documents\\COMP0190\\Data\\COMP0191-MSc-Project-Code\\Stata-Results\\UKHLS_demo_responses.dta"

stata_demo_question_responses = load_stata_file(stata_demo_question_filepath)

stata_demo_question_responses_wave_eight, stata_demo_question_responses_wave_nine, stata_demo_question_responses_wave_ten = [stata_demo_question_responses[stata_demo_question_responses["wavename"] == wavename] for wavename in [8.0, 9.0, 10.0]]

stata_demo_question_responses_wave_ten_preprocessed = preprocess_stata_data(stata_demo_question_responses_wave_ten)

print(stata_demo_question_responses_wave_ten_preprocessed)

           sex   marstat               qfhigh racel_dv lnprnt age_dv  \
138043    male   Married     Higher education  British    1.0   55.0   
138044  female   Married  Secondary education  British    3.0   57.0   
138045  female   Married         inapplicable  British    3.0   32.0   
138046    male   Married  Secondary education  British    1.0   33.0   
138049    male  Divorced  Secondary education  British    1.0   61.0   
...        ...       ...                  ...      ...    ...    ...   
159408    male   Married  Secondary education   Indian    1.0   77.0   
159409  female   Married  Secondary education   Indian    2.0   46.0   
159410    male   Married  Secondary education   Indian    6.0   49.0   
159411  female    Single  Secondary education   Indian    1.0   19.0   
159412  female    Single  Secondary education   Indian    1.0   17.0   

            gor_dv    urban_dv         agegr10_dv  \
138043  North East  urban area    50-59 years old   
138044  North East  urban are

In [4]:
print(stata_demo_question_responses_wave_ten_preprocessed["scenv_crlf"].value_counts())

I do quite a few things that are environmentally-friendly      4743
I do one or two things that are environmentally-friendly       4045
I'm environmentally-friendly in most things I do               1928
I don't really do anything that is environmentally-friendly     684
I'm environmentally-friendly in everything I do                 266
missing                                                           0
inapplicable                                                      0
proxy                                                             0
refusal                                                           0
don't know                                                        0
Name: scenv_crlf, dtype: int64


In [5]:
# Define the list of acceptable response options
demo_response_options = [
    "I don't really do anything that is environmentally-friendly",
    "I do one or two things that are environmentally-friendly",
    "I do quite a few things that are environmentally-friendly",
    "I'm environmentally-friendly in most things I do",
    "I'm environmentally-friendly in everything I do"
]

# split the data by responses: "Don't do Anything Environmentally Friendly", "Do One or Two Things Environmentally Friendly", "Do Some Things Environmentally Friendly", "Do Many Things Environmentally Friendly", "Do Everything Environmentally Friendly"
dont_do_anything_responses = stata_demo_question_responses_wave_ten_preprocessed[stata_demo_question_responses_wave_ten_preprocessed['scenv_crlf'] == "I don't really do anything that is environmentally-friendly"]

do_one_or_two_things_responses = stata_demo_question_responses_wave_ten_preprocessed[stata_demo_question_responses_wave_ten_preprocessed['scenv_crlf'] == "I do one or two things that are environmentally-friendly"]

do_some_things_responses = stata_demo_question_responses_wave_ten_preprocessed[stata_demo_question_responses_wave_ten_preprocessed['scenv_crlf'] == "I do quite a few things that are environmentally-friendly"]

do_many_things_responses = stata_demo_question_responses_wave_ten_preprocessed[stata_demo_question_responses_wave_ten_preprocessed['scenv_crlf'] == "I'm environmentally-friendly in most things I do"]

do_everything_responses = stata_demo_question_responses_wave_ten_preprocessed[stata_demo_question_responses_wave_ten_preprocessed['scenv_crlf'] == "I'm environmentally-friendly in everything I do"]

In [6]:
def write_balanced_responses_to_json(input_data, variable_name):

    num_rows = input_data.shape[0]

    print(num_rows)
    
    unique_indices = random.sample(range(num_rows), 10)

    # generate 10 random responses for each response type

    for random_index in unique_indices:
        random_row = input_data.iloc[random_index]
        profile_string, opinion_string = visualise_ukhls_profile_and_response(random_row, variable_name)

        if variable_name == "scenv_crlf":

            demo_json_filepath = "C:\\Users\\haoch\\Documents\\COMP0190\\Data\\COMP0191-MSc-Project-Code\\Stata-Results\\stata_demo_question_responses_wave_ten_demo.json"

            # write the profile and opinion to a json file
            with open(demo_json_filepath, "r") as file:
                output_data = json.load(file)

            output_data["profiles"].append(profile_string)
            output_data["opinions"].append(opinion_string)
            
            with open(demo_json_filepath, "w") as file:
                json.dump(output_data, file, indent=4)

In [8]:
write_balanced_responses_to_json(dont_do_anything_responses, "scenv_crlf")
write_balanced_responses_to_json(do_one_or_two_things_responses, "scenv_crlf")
write_balanced_responses_to_json(do_some_things_responses, "scenv_crlf")
write_balanced_responses_to_json(do_many_things_responses, "scenv_crlf")
write_balanced_responses_to_json(do_everything_responses, "scenv_crlf")

684
4045
4743
1928
266
